# Regression, Instrumental Variables and Regression Discontinuity

This notebook illustrates how to do regression analysis Python with Pandas.
Original dataset available at: http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets (search for rhs, download csv file for data, html file for file description)
For a presentation and key results on the topic, see: http://www.mc.vanderbilt.edu/crc/workshop_files/2008-04-11.pdf

## Import key modules

In [1]:
from __future__ import division
import pandas as pd
from pandas import DataFrame, Series
from collections import Counter
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


/opt/tljh/user/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/tljh/user/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/tljh/user/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Import data to a dataframe (called df)

In [3]:
df = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.csv')

## Have a look at the data

In [4]:
df.head(5)

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


## Clean the Data

In [6]:
df=df.replace('Yes', 1)
df=df.replace('No', 0)

df.swang1=df.swang1.replace('No RHC', 0)
df.swang1=df.swang1.replace('RHC', 1)

df.sex=df.sex.replace('Male', 0)
df.sex=df.sex.replace('Female', 1)

In [7]:
df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,1,11142,11151.0,NaN,11382,0,0,...,0,0,0,0,0,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,0,11799,11844.0,11844.0,11844,1,1,...,0,0,1,0,0,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,1,12083,12143.0,NaN,12400,0,0,...,0,0,0,0,0,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,0,11146,11183.0,11183.0,11182,1,0,...,0,0,0,0,0,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,0,12035,12037.0,12037.0,12036,1,0,...,0,0,0,0,0,NaN,64.0,white,Under $11k,11


In [8]:
df.swang1.head()

0    0
1    1
2    1
3    0
4    1
Name: swang1, dtype: int64

In [9]:
# percent dead in different groups
df.groupby('swang1')['death'].mean()

swang1
0    0.629682
1    0.680403
Name: death, dtype: float64

In [9]:
df.columns


Index([u'cat1', u'cat2', u'ca', u'sadmdte', u'dschdte', u'dthdte', u'lstctdte', u'death', u'cardiohx', u'chfhx', u'dementhx', u'psychhx', u'chrpulhx', u'renalhx', u'liverhx', u'gibledhx', u'malighx', u'immunhx', u'transhx', u'amihx', u'age', u'sex', u'edu', u'surv2md1', u'das2d3pc', u't3d30', u'dth30', u'aps1', u'scoma1', u'meanbp1', u'wblc1', u'hrt1', u'resp1', u'temp1', u'pafi1', u'alb1', u'hema1', u'bili1', u'crea1', u'sod1', u'pot1', u'paco21', u'ph1', u'swang1', u'wtkilo1', u'dnr1', u'ninsclas', u'resp', u'card', u'neuro', u'gastr', u'renal', u'meta', u'hema', u'seps', u'trauma', u'ortho', u'adld3p', u'urin1', u'race', u'income', u'ptid'], dtype='object')

## Run regression

In [17]:
model = 'death ~ age + sex + edu + rhc'
reg_results = smf.ols(formula=model, data=df).fit()
reg_results.summary()

NameError: name 'smf' is not defined

In [10]:
outcome = df.death
renameVars = {'swang1': 'rhc'}
df.rename(columns = renameVars, inplace = True)
vars = ['age', 'sex', 'edu', 'rhc']

olsResult = sm.OLS(outcome, df[vars]).fit()
print (olsResult.params)

age    0.008125
sex   -0.012853
edu    0.010540
rhc    0.067150
dtype: float64


In [11]:
print (olsResult.summary())


                            OLS Regression Results                            
Dep. Variable:                  death   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     2829.
Date:                Mon, 29 Oct 2018   Prob (F-statistic):               0.00
Time:                        23:32:29   Log-Likelihood:                -3772.0
No. Observations:                5735   AIC:                             7552.
Df Residuals:                    5731   BIC:                             7579.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.0081      0.000     31.665      0.0

In [13]:
outcome = df.death
renameVars = {'swang1': 'rhc'}
df.rename(columns = renameVars, inplace = True)
vars = ['age', 'sex', 'edu', 'rhc']

logitResult = sm.Logit(outcome, df[vars]).fit()
print(logitResult.summary())

Optimization terminated successfully.
         Current function value: 0.627138
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  death   No. Observations:                 5735
Model:                          Logit   Df Residuals:                     5731
Method:                           MLE   Df Model:                            3
Date:                Mon, 29 Oct 2018   Pseudo R-squ.:                 0.03229
Time:                        23:32:51   Log-Likelihood:                -3596.6
converged:                       True   LL-Null:                       -3716.7
                                        LLR p-value:                 9.368e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.0192      0.001     15.885      0.000       0.017       0.022
sex           -0.1585      0.

In [50]:
groups['death'].size().unstack(level=0)

swang1,0,1
1,582,248
2,1330,833
3,1246,877
4,392,226
5,1,NaN


In [51]:
psTable = groups['death'].mean().unstack(level=0)
psTable

swang1,0,1
1,0.707904,0.758065
2,0.669173,0.721489
3,0.609149,0.659065
4,0.443878,0.526549
5,1.000000,NaN


In [52]:
psTable.columns = ['untreated', 'treated']

In [53]:
psTable['difference'] = psTable.treated - psTable.untreated

In [55]:
psTable.tail(5)

,untreated,treated,difference
1,0.707904,0.758065,0.050161
2,0.669173,0.721489,0.052316
3,0.609149,0.659065,0.049916
4,0.443878,0.526549,0.082671
5,1.000000,NaN,NaN


In [56]:
psTable.difference.mean()

0.058765809330939417